# 🚀 AG2 Foundations: Tools

Welcome! This notebook walks you through the foundations of using **tools** in the AG2 (AutoGen) framework.

Tools are a core part of how agents take meaningful actions — from doing math to making API calls, searching the web, or extracting knowledge from text. This notebook is structured to guide you step-by-step, with each section building on the last.

---

### 📚 What You’ll Learn

- How to define and register your own tools  
- How to inject secrets into tools safely  
- How to integrate tools from other frameworks
- How to use AG2's built-in reference tools  
- How to orchestrate multiple tools in a full research workflow

Let’s dive in and start building intelligent, tool-using agents!

**Note:** For this demo you will need a few extra dependcies:

```bash
pip install ag2[openai,interop-langchain,google-search]
pip install langchain_community
pip install arxiv

### ⚙️ LLM Configuration

Make sure to use a model that supports structured tool calling and has strong reasoning capabilities, so the agent can correctly decide which tools to call and when.



In [51]:
from autogen import LLMConfig

llm_config = LLMConfig(api_type="openai", model="gpt-4o")

### 🧠 Tool Usage Between Agents: Simple Math Example

In this example, we define a minimal tool (`add_numbers`) and demonstrate how two agents can collaborate to use it.

- **The tool:** A basic function that adds two integers.
- **The addition agent** acts as the *caller* — it has reasoning capabilities and decides when to use the tool based on the user's question.
- **The executor agent** is the *executor* — it doesn't talk to the user but runs the actual function when instructed.
- We use `register_function` to make the tool available to the agents.
- Finally, the user asks a question like "What is 2 + 2?", and the `addition_agent` will use the function to help answer.

In [ ]:
from autogen import ConversableAgent, register_function

# 1. Define a function
def add_numbers(a: int, b: int) -> int:
    return a + b

# 2. Create an agent that will decide when to use the tool (the "caller")
addition_agent = ConversableAgent(
    name="addition_agent",
    llm_config=llm_config,
    system_message="You are a helpful math assistant that can only do addition. reply in this format 'x + y = z'", 
)

# 3. Create another agent that will execute the tool (the "executor")
executor_agent = ConversableAgent(
    name="executor",
    human_input_mode="NEVER",  
    is_termination_msg=lambda x: "=" in (x.get("content", "") or "").upper() # terminate chat when input has '=' 
)

# 4. Register the tool function with both agents
register_function(
    add_numbers,
    caller=addition_agent,    
    executor=executor_agent,   
    description="Add two numbers together",  # description helps the LLM know what the tool does and when to use it
)

# 5. Ask a question that requires the tool
result = executor_agent.initiate_chat(
    recipient=addition_agent,
    message="What is 2 + 2?",
    # max_turns=2  
)

### 📡 Tool with Secrets

Secrets such as password, tokens, or personal information needs to be protected from capture. AG2 provides dependency injection as a way to secure this sensitive information while still allowing agents to perform their tasks effectively, even when working with large language models (LLMs).

**Benefits of dependency injection:**

- **Enhanced Security:** Your sensitive data is never directly exposed to the LLM or telemetry.  
- **Simplified Development:** Secure data can be seamlessly accessed by functions without requiring complex configurations.  
- **Unmatched Flexibility:** Supports safe integration of diverse workflows, allowing you to scale and adapt with ease.

In this walk-through we’ll show how you could support 3rd party system credentials using specific agents and their respective tools and dependency injection.


In [ ]:
from typing import Annotated
from pydantic import BaseModel
from autogen import ConversableAgent, register_function
from autogen.tools.dependency_injection import BaseContext, Depends

# Mock weather API function (pretending to hit a third-party API)
def weather_api_call(username: str, password: str, location: str) -> str:
    print(f"Accessing third party Weather System using username {username}")
    return "It's sunny and 40 degrees Celsius in Sydney, Australia."

# Define a context class to store secret credentials
class ThirdPartyCredentials(BaseContext, BaseModel):
    username: str
    password: str

weather_account = ThirdPartyCredentials(username="ag2weather", password="wbkvEehV1A")

# function that receives location input and gets credentials injected
def get_weather(
    location: str,
    credentials: Annotated[ThirdPartyCredentials, Depends(weather_account)],
) -> str:
    # Access the Weather API using the credentials
    return weather_api_call(username=credentials.username, password=credentials.password, location=location)

weather_agent = ConversableAgent(
    name="weather_agent",
    llm_config=llm_config,
    system_message="You are a weather assistant."
)

executor_agent = ConversableAgent(name="executor", human_input_mode="NEVER")

register_function(
    get_weather,
    caller=weather_agent,
    executor=executor_agent,
    description="Gets weather for given location"
)

query = "Get weather for Sydney, Australia"
result = executor_agent.initiate_chat(recipient=weather_agent, message=query, max_turns=2)

### 🔄 Tool Interoperability 

AG2 supports tool interoperability with popular agent frameworks like **LangChain**, **CrewAI**, and **Pydantic**. Using AG2’s `Interoperability` module, you can convert tools from these ecosystems into AG2-compatible functions with just one line of code.

In this demo, we:

- Use a LangChain-based ArXiv tool
- Convert it into an AG2-compatible tool 
- Register and use it inside a `ConversableAgent` just like any native tool

This makes it easy to **reuse existing tooling** and integrate with external agent stacks without rewriting functionality.


In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

from autogen.interop import Interoperability

# Initialize LangChain Wikipedia tool
api_wrapper = ArxivAPIWrapper()
langchain_tool = ArxivQueryRun(api_wrapper=api_wrapper)


# Convert to AG2 tool
interop = Interoperability()
ag2_arXiv_tool = interop.convert_tool(tool=langchain_tool, type="langchain")


arxiv_agent = ConversableAgent(
    name="arxiv_agent",
    llm_config=llm_config,
    system_message="You are helpful ai assistant"
)

executor_agent = ConversableAgent(name="executor", human_input_mode="NEVER")

# Register the converted LangChain tool just like any other function
register_function(
    ag2_arXiv_tool,
    caller=arxiv_agent,
    executor=executor_agent,
    description="Search and retrieve academic papers from ArXiv."
)


query = "Tell me about llm prompting"
result = executor_agent.initiate_chat(
    recipient=arxiv_agent, 
    message=query, 
    max_turns=2
)

### 🌐 Reference Tools in AG2

In this section, we'll demonstrate how to enhance an AG2 agent's capabilities with reference tools by integrating the **Google Search Tool**. This tool enables the agent to perform real-time web searches, providing up-to-date information beyond its training data.

**Prerequisites:**

- **Google Search API Key:** Obtain from the [Google Cloud Console](https://console.cloud.google.com/).
- **Search Engine ID:** Set up a custom search engine via the [Google Programmable Search Engine](https://programmablesearchengine.google.com/).

**Note:** You will need to install `ag2[google]` and set environment variables for `GOOGLE_SEARCH_API_KEY` and `GOOGLE_SEARCH_ENGINE_ID`.

---

- Load the reference `GoogleSearchTool` from AG2.
- Configures it with API credentials.
- Registers it as a tool callable by a `ConversableAgent`.
- Executes a web search using the agent.

This shows how AG2's **reference tools** let you extend agent capabilities with minimal setup.


In [ ]:
import os
from autogen import ConversableAgent, register_function
from autogen import AssistantAgent, LLMConfig
from autogen.tools.experimental import GoogleSearchTool

# Load Google Search API credentials from environment variables
search_api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

# Initialize the Google Search tool with your API key and search engine ID
gs_tool = GoogleSearchTool(
    search_api_key=search_api_key,
    search_engine_id=search_engine_id,
)

gs_agent = ConversableAgent(
    name="gs_agent",
    llm_config=llm_config,
    system_message="You are a helpful ai assistant."
)

executor_agent = ConversableAgent(name="executor", human_input_mode="NEVER")

register_function(
    gs_tool,
    caller=gs_agent,
    executor=executor_agent,
    description="Search the web using the Google Search API to retrieve real-time information."
)

query = "Who is the founder of AG2"
result = executor_agent.initiate_chat(
    recipient=gs_agent, 
    message=query, 
    max_turns=2
)

### 🧠 ML Research Assistant: Multi-Tool Orchestration

In this final demo, we combine everything you've learned into a single, powerful agent.

The `ml_research_agent` has access to three tools:
- 🧪 **Custom Function** – Method extraction tool for identifying ML techniques like GPT, Transformer, BERT, etc.
- 📄 **Arxiv Search Tool** – for academic papers and abstracts
- 🌐 **Google Search Tool** – for real-time web results

Unlike earlier examples where the agent had just one tool, this agent now has access to multiple tools and must reason about **when and how to use each one**.  
This is where **clear system messages and strong tool descriptions** become essential — guiding the agent to make correct decisions in multi-step workflows.

<img src="diagram.png" alt="Alt text" width="400"/>

In [ ]:
from typing import List

def extract_methods(text: str) -> List[str]:
    methods = ["Transformer", "LLM", "CNN", "RNN", "BERT", "GPT", "LSTM", "Reinforcement Learning", "Diffusion", "Autoencoder"]
    return [m for m in methods if m.lower() in text.lower()]


ml_research_agent = ConversableAgent(
    name="research_agent",
    llm_config=llm_config,
    system_message= 
    '''
    You are a machine learning research assistant with access to three tools: web search, Arxiv paper search, and method extraction.

    Your task is to:
    1. Use the Arxiv and Google Search tools to retrieve context on query.
    2. After you have context use the method extraction tool.
    3. After you retrieved context and extracted terms then summarize and list extracted terms. Conclude the research session by responding with 'TERMINATE'.

    Note: Use each tool no more than once during the entire research process.
    '''
)

executor_agent = ConversableAgent(
    name="executor",
    human_input_mode="NEVER"
)

register_function(
    extract_methods,
    caller=ml_research_agent,
    executor=executor_agent,
    description="After retireving context this tool identifies and returns relevant terms from text (e.g., Transformer, GPT, CNN)."
)

register_function(
    ag2_arXiv_tool,
    caller=ml_research_agent,
    executor=executor_agent,
    description="Search and retrieve scientific papers and abstracts from arXiv."
)

register_function(
    gs_tool,
    caller=ml_research_agent,
    executor=executor_agent,
    description="Search the web using the Google Search API for up-to-date research or factual information."
)

query = "How does chatgpt work"
result = executor_agent.initiate_chat(
    recipient=ml_research_agent, 
    message=query, 
    max_turns=10
)